# Aprendizado Supervisionado

Importando dependências

In [2]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyRegressor
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, accuracy_score

---
## Funções auxiliares

### Função para carregar os *DataFrames*
Dado o ano, mês, número de meses anteriores e o número de agrupamentos, retorna uma tupla de dataframes de treino e teste

In [2]:
def load_dataframe(year, month, n_months):
    dirname = 'df_dynamic'
    df_train = pd.read_pickle('{0}/{1}_{2}_{3}_train.pkl'.format(dirname, year, month, n_months))
    df_test = pd.read_pickle('{0}/{1}_{2}_{3}_test.pkl'.format(dirname, year, month, n_months))
    return df_train, df_test

**Exemplo:** 3 meses anteriores a Fevereiro de 2019 com 2000 agrupamentos retorna uma tupla com:
* *DataFrame* de treino com os dados dos meses de Novembro de 2018, Dezembro de 2018 e Janeiro de 2019
* *DataFrame* de teste com os dados do mês de Fevereiro de 2019

In [3]:
df_train, df_test = load_dataframe(2019, 2, 3)

In [4]:
df_train

,DATAOCORRENCIA,HORAOCORRENCIA,BAIRRO,CIDADE,LATITUDE,LONGITUDE,MES,GRUPO
1,31/10/2018,21:30,JD IRMÃOS SIGRIST,CAMPINAS,-23.003776,-47.106153,1,628
4,31/10/2018,21:00,JD ARARUAMA,COTIA,-23.609402,-46.929926,1,911
5,31/10/2018,23:00,SAO LUCAS,S.PAULO,-23.576057,-46.554417,1,3639
6,31/10/2018,23:00,SAO RAFAEL,S.PAULO,-23.630611,-46.471524,1,4143
7,31/10/2018,23:40,BRASILANDIA,S.PAULO,-23.457706,-46.690668,1,3839
...,...,...,...,...,...,...,...,...
22556,31/01/2019,22:20,SAO MATEUS,S.PAULO,-23.614637,-46.470736,3,4014
22557,31/01/2019,22:00,ITAIM BIBI,S.PAULO,-23.586411,-46.676220,3,4370
22559,31/01/2019,20:05,CIDADE TIRADENTES,S.PAULO,-23.587221,-46.406488,3,3686
22560,31/01/2019,20:30,TREMEMBE,S.PAULO,-23.452332,-46.601216,3,4374


In [5]:
df_test

,DATAOCORRENCIA,HORAOCORRENCIA,BAIRRO,CIDADE,LATITUDE,LONGITUDE,MES,GRUPO
0,31/01/2019,19:00,ITAIM PAULISTA,S.PAULO,-23.510144,-46.397749,4,4009
1,31/01/2019,21:45,VALO VELHO,ITAPECERICA DA SERRA,-23.689874,-46.800888,4,1615
2,31/01/2019,16:20,CAETETUBA,ATIBAIA,-23.114819,-46.586959,4,198
4,28/01/2019,04:30,ITAIM PAULISTA,S.PAULO,-23.497685,-46.393311,4,4226
5,31/01/2019,19:30,BOM RETIRO,S.PAULO,-23.524828,-46.630724,4,3592
...,...,...,...,...,...,...,...,...
20807,28/02/2019,20:45,GUAIANASES,S.PAULO,-23.542405,-46.419923,4,4393
20808,28/02/2019,19:00,SANTO AMARO,S.PAULO,-23.628584,-46.691692,4,2000
20809,28/02/2019,16:00,VILA SONIA,S.PAULO,-23.595742,-46.740399,4,3709
20810,28/02/2019,22:40,VILA HELENA II,SUZANO,-23.597856,-46.307167,4,4868


### Função para obter número de agrupamentos
Dado o *DataFrame* de treinamento, retorna o número de agrupamentos

In [6]:
def get_n_clusters(df_train):
    n_clusters, _ = df_train[['BAIRRO', 'CIDADE', 'LATITUDE', 'LONGITUDE']].groupby(['CIDADE', 'BAIRRO']).mean().dropna().to_numpy().shape
    return n_clusters

In [7]:
get_n_clusters(df_train)

5111

### Função para transformar os atributos do *DataFrame*
Dado o *DataFrame* e o número de agrupamentos, a função cria uma série temporal com os números de eventos mensais de cada grupo e retorna uma tupla com os dados dos atributos e valores-alvo.

In [8]:
def get_X_y(df, n_clusters):
    
    # cria uma série temporal com o número de eventos mensais de cada grupo
    df = df.groupby(['GRUPO', 'MES']).size().reset_index()
    df.columns = ['GRUPO', 'MES', 'COUNT']
    
    # transforma o número de cada grupo em variáveis categóricas
    pipeline = ColumnTransformer([
        ('grupo', OneHotEncoder(categories=[np.arange(n_clusters, dtype='int32')]), ['GRUPO']),
        ('mes', 'passthrough', ['MES'])
    ])
    X = pipeline.fit_transform(df)
    y = df['COUNT']
    return X, y

**Exemplo:** Transformando os atributos dos *DataFrames* de treino e teste

In [9]:
n_clusters = get_n_clusters(df_train)
X_train, y_train = get_X_y(df_train, n_clusters)
X_test, y_test = get_X_y(df_test, n_clusters)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((10334, 5112), (10334,), (3200, 5112), (3200,))

---
## Funções para Métricas de Desempenho

### Função para obter a Acurácia e Medida F<sub>1</sub> do Modelo de Regressão
Dado um modelo de regressão, ano, mês e número de meses anteriores, retorna uma tupla com Acurácia e a Medida $F_1$ do modelo.

In [1]:
def regressor_scores(regressor, year, month, n_month):
    
    # carregamento dos dataframes de treino e teste
    df_train, df_test = load_dataframe(year, month, n_month)
    n_clusters = get_n_clusters(df_train)
    
    # transformação dos atributos
    X_train, y_train = get_X_y(df_train, n_clusters)
    X_test, y_test = get_X_y(df_test, n_clusters)

    # treinamento e predição do modelo de regressão
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)

    # categorização do resultado da predição e teste, baseado na mediana
    threshold = y_train.median()
    h_pred = y_pred >= threshold
    h_test = y_test >= threshold

    # retorna a Acurácia e Medida F1
    return accuracy_score(h_test, h_pred), f1_score(h_test, h_pred, average='weighted')

**Exemplo:** Obtendo a Acurácia e a Medida F<sub>1</sub> do modelo de Regressão Linear com os dados de 3 meses anteriores a Fevereiro de 2019

In [11]:
lr = LinearRegression()
regressor_scores(lr, 2019, 2, 3)

(0.7428125, 0.7489214467899266)

### Função para obter a Acurácia e Medida F<sub>1</sub> do Algoritmo de Classificação
Dado um modelo de classificação, ano, mês e número de meses anteriores, retorna uma tupla com Acurácia e a Medida F<sub>1</sub> do modelo.

In [2]:
def classifier_scores(classifier, year, month, n_month):
    
    # carregamento dos dataframes de treino e teste
    df_train, df_test = load_dataframe(year, month, n_month)
    n_clusters = get_n_clusters(df_train)
    
    # transformação dos atributos
    X_train, y_train = get_X_y(df_train, n_clusters)
    X_test, y_test = get_X_y(df_test, n_clusters)
    
    # categorização das variáveis-alvo baseado na mediana
    threshold = y_train.median()
    h_train = y_train >= threshold
    h_test = y_test >= threshold

    # treinamento e predição do modelo de classificação
    classifier.fit(X_train, h_train)
    h_pred = classifier.predict(X_test)
    
    # retorna Acurácia e Medida F1
    return accuracy_score(h_test, h_pred), f1_score(h_test, h_pred, average='weighted')

**Exemplo:** Obtendo a Acurácia e a Medida F<sub>1</sub> do modelo de Regressão Logística com os dados de 3 meses anteriores a Fevereiro de 2019

In [13]:
lrc = LogisticRegression()
classifier_scores(lrc, 2019, 2, 3)

(0.7359375, 0.730754739856343)

---
## Testes

### Função para guardar os dados de desempenho
Dado a função de métrica de desempenho, modelo e o nome do diretório, realiza a bateria de testes para o ano de 2019 e guarda os resultados no diretório definido.

In [14]:
def save_scores(scores, model, dirname):
    
    # criação da pasta para guardar os resultados
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    # ano, meses, número de meses anteriores e número de agrupamentos
    year = 2019
    months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    n_months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
        
    # estruturas de dados para as métricas de desempenho
    accuracy_scores = np.zeros((len(months), len(n_months)))
    f1_scores = np.zeros((len(months), len(n_months)))
    elapsed_time = np.zeros((len(months), len(n_months)))
    
    # para cada mês e número de meses anteriores:
    for x, month in enumerate(months):
        for y, n_month in enumerate(n_months):
            start = time.time()
            # atribuímos as medidas de desempenho à estrutura de dados
            accuracy_scores[x][y], f1_scores[x][y] = scores(model, year, month, n_month)
            stop = time.time()
            elapsed_time[x][y] = stop - start
                
    # por fim, salvamos os resultados atribuídos na estrutura de dados
    np.save('{0}/{1}_accuracy'.format(dirname, year), accuracy_scores)
    np.save('{0}/{1}_f1'.format(dirname, year), f1_scores)
    np.save('{0}/{1}_time'.format(dirname, year), elapsed_time)

### Testes dos Regressores

Modelo Base (somente média)

In [ ]:
dr = DummyRegressor(strategy='mean')
save_scores(regressor_scores, dr, 'scores_dynamic_dr')

Regressão Linear

In [16]:
lr = LinearRegression(n_jobs=-1)
save_scores(regressor_scores, lr, 'scores_dynamic_lr')

Regressão com *Random Forest*

In [17]:
rfr = RandomForestRegressor(n_jobs=-1)
save_scores(regressor_scores, rfr, 'scores_dynamic_rfr')

Regressão com *Perceptron* de Múltiplas Camadas

In [21]:
mlpr = MLPRegressor(early_stopping=True, max_iter=100000)
save_scores(regressor_scores, mlpr, 'scores_dynamic_mlpr')

### Testes dos Classificadores

Modelo Base (classe de maior frequência)

In [18]:
dc = DummyClassifier(strategy='prior')
save_scores(classifier_scores, dc, 'scores_dynamic_dc')

Regressão Logística

In [19]:
lrc = LogisticRegression(n_jobs=-1)
save_scores(classifier_scores, lrc, 'scores_dynamic_lrc')

Classificador com *Random Forest*

In [20]:
rfc = RandomForestClassifier(n_jobs=-1)
save_scores(classifier_scores, rfc, 'scores_dynamic_rfc')

Classificador com *Perceptron* de Múltiplas Camadas

In [22]:
mlpc = MLPClassifier(early_stopping=True, max_iter=100000)
save_scores(classifier_scores, mlpc, 'scores_dynamic_mlpc')